# Defining an ODE system

The starting point in using PyGOM to study an ODE system is to encapsulate the relevant information about the system into a class {class}`SimulateOde`.
When set up in this object, the system is prepared for application of the various features of PyGOM such as simulation and model fitting.

<!-- To ODE system into a class {class}`Transition` -->
## Equations vs Transitions

Firstly, we outline the two main ways we might think about defining our ODE system.
Typically, we may already know the set of equations which describes the rates of change of the dependent variables:

$$\frac{\mathrm{d} \mathbf{y}}{\mathrm{d} t} = f(\mathbf{y},\boldsymbol{\theta}, t)$$

where $\mathbf{y} = \left(y_{1},\ldots,y_{n}\right)$ is the state vector, $\boldsymbol{\theta} = \left(\theta_{1},\ldots, \theta_{p}\right)$ is the set of parameters and $f$ is a vector function giving the rates of change of each state.

Compartmental models, whereby the variables represent categories, have another interpretation in which we may instead consider transitions between the different groups

$$\begin{aligned}
y_i \rightarrow y_j = f_{i,j}(\mathbf{y},\boldsymbol{\theta}) \\
\end{aligned}$$

where $i, j = \{1,\ldots, n\}$ and $f_{i,j}$ are functions governing the rate at which members of group $i$ transition to $j$.

PyGOM allows the user flexibility to choose which of these perspectives to use, or indeed combine, in order to build their models.
If given transitions, Pygom will automatically convert these into equations and, as we'll see later, it can also attempt to reverse engineer transitions from equations.

## Example: SIR model

Here we use a Susceptible-Infected-Recovered epidemic model (an SIR model, for short) to demonstrate the two different ways PyGOM supports model specification.
Since we will frequently rely on the SIR model in examples throughout the tutorial, we take a moment here to outline its key features.

Firstly, there is some ambiguity in the naming of the Recovered class which is also commonly referred to as Removed.
In the latter sense, there is no distinction made between those who can no longer be infected due to infection acquired immunity and infection induced death.
However, for more complex models, the recovered class may become susceptible again due to immune waning and number of deaths is typically important to distinguish.
Thus, we make the distinction clear from the outset so that when we talk about deaths in this tutorial it will have its own class, Dead, separate from Recovered. 

The assumptions of the SIR model:
1) An average member of the population makes contact sufficient to transmit infection with $\beta$ others per unit time (*standard* as opposed to *mass-action* incidence, where this would be $\beta N$).
2) Contacts are well mixed throughout the population, so that the probability that a given contact is with an infected person is $\frac{I}{N}$ where $N$ is the total population size.
3) The infective class is depleted through recovery at a rate $\gamma I$.
4) No births, deaths (natural or from disease) or migration mean there is no entry into or departure from the population: $S(t)+I(t)+R(t)=N$.

Under these assumptions, the rates of change of the population in each compartment (**S**usceptible, **I**nfected and **R**ecovered) are given by the following equations:

$$\begin{aligned}
\frac{\mathrm{d} S}{\mathrm{d} t} &= -\frac{\beta SI}{N} \\
\frac{\mathrm{d} I}{\mathrm{d} t} &= \frac{\beta SI}{N} - \gamma I \\
\frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I.
\end{aligned}$$

However, it is equally valid to view this system as the result of two transitions; infection, which takes subjects from susceptible to infected and recovery, which takes them from infected to recovered.

$$\begin{aligned}
S \rightarrow I &= \frac{\beta SI}{N} \\
I \rightarrow R &= \gamma I
\end{aligned}$$

Advantages of specifying our system using the transition approach are firstly that transitions are a more intuitive language when thinking about building compartmental models.
Perhaps more beneficial, though, is that it enables the computer to do our book-keeping when converting transitions to ODE equations.
This reduces the error of, for example, including a flow out of one state, but forgetting to include it in the recipient state.
In this case, we had to remember to include $-\frac{\beta SI}{N}$ in the $S$ equation and the opposite sign in the $I$ equation, for example.

## Defining the model with PyGOM

Defining the system is handled by feeding classes of {class}`Transition` into the main class we referred to at the start, {class}`SimulateOde`.
The {class}`Transition` class has 4 types, which are specified by the `transition_type` argument.
This lets `SimulateOde` know in which format we are inputting our information.
This could be as transitions or ODEs as we've just been discussing, or as the other two available options which are birth and death processes.
The {class}`Transition` class accepts 3 or 4 parameters depending on which type is being defined as summarised in the below table:

|                 |          Transition             |            Equation               |                  Birth                 |                  Death                 |
|:---------------:|:-------------------------------:|:---------------------------------:|:--------------------------------------:|:--------------------------------------:|
| transition_type | T                               | ODE                               | B                                      | D                                      |
| origin          | State transition is from, $y_i$ | State equation pertains to, $y_i$ | State birth process populates, $y_i$   | State death process depletes, $y_i$    |
| destination     | State transition is to, $y_j$   | n/a                               | n/a                                    | n/a                                    |
| equation        | $f_{i,j}$                       | $\frac{dy_i}{dt}$                 | $\frac{dy_i}{dt}$ due to birth process | $\frac{dy_i}{dt}$ due to death process |

```{note}
Arguably the state which the birth process populates could be defined as a destination rather than an origin, but this is the convention adopted here.

Also, the {class}`Transition` class may have been better named.
Firstly, one of the four possible input types is also called "transition", which would imply that it has a special place within the class given that it shares its name.
However, all other forms of input (ODE, birth and death) are equally valid.
```

Let's see how this is implemented for our example SIR system.

In [ ]:
from pygom import Transition, TransitionType

# Define SIR model through a list of ODEs
odeList = [
    Transition(transition_type=TransitionType.ODE, origin='S', equation='-beta*S*I'),
    Transition(transition_type=TransitionType.ODE, origin='I', equation='beta*S*I - gamma*I'),
    Transition(transition_type=TransitionType.ODE, origin='R', equation='gamma*I') 
]

# Define SIR model through a list of transitions
transList = [
    Transition(transition_type=TransitionType.T, origin='S', destination='I', equation='beta*S*I'),
    Transition(transition_type=TransitionType.T, origin='I', destination='R', equation='gamma*I')
]

We now initialise two `SimulateODE` objects using these different approaches.
In addition to the ODE or transition information, we must specify which variables are parameters and which refer to states.

In [ ]:
stateList = ['S', 'I', 'R']
paramList = ['beta', 'gamma']

We define the model in two different ways

In [ ]:
from pygom import SimulateOde
model_ode = SimulateOde(state=stateList, param=paramList, ode=odeList)           # model defined via equations
model_tra = SimulateOde(state=stateList, param=paramList, transition=transList)  # model defined via transitions

We can use the `get_ode_eqn` function to verify that the models are equivalent

In [ ]:
model_ode.get_ode_eqn()

In [ ]:
model_tra.get_ode_eqn()

## Modifying models

Once a `SimulateODE` object has been defined, it may be modified.
For example, let's say we wish to add birth and death processes to the the previously defined SIR model of `model_ode`.
We must update the parameters of the class to reflect any additions.

In [ ]:
model_ode.param_list = model_ode.param_list + ['mu', 'B']

We must also add in the new processes

In [ ]:
birthDeathList = [Transition(origin='S', equation='B', transition_type=TransitionType.B),  
                  Transition(origin='S', equation='mu*S', transition_type=TransitionType.D), 
                  Transition(origin='I', equation='mu*I', transition_type=TransitionType.D), 
                  Transition(origin='R', equation='mu*R', transition_type=TransitionType.D)]

model_ode.birth_death_list = birthDeathList

We can again use `get_ode_eqn` to verify that the equations are updated

In [ ]:
model_ode.get_ode_eqn()